In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\azure-vm\\Repo\\azure-vm")

import datasets

dataset = datasets.load_dataset("Clinton/Text-to-sql-v1")
df = dataset["train"].to_pandas()

df = df[df['source'].isin(['sql_create_context', 'spider', 'mimicsql_data'])]
df.head()

df_copy = df.copy()


def add_length_column(df):
    df["total_length"] = 0
    for column_name in ["instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

def filter_by_join_count(df, difficulty, number_of_samples):
    if difficulty == "lvl1":
        join_mask = df["response"].str.count("JOIN") == 0
    elif difficulty == "lvl2":
        join_mask = df["response"].str.count("JOIN") == 1
    elif difficulty == "lvl3":
        join_mask = df["response"].str.count("JOIN") == 2
    elif difficulty == "lvl4":
        join_mask = df["response"].str.count("JOIN") >= 3
    else:
        raise ValueError("Invalid difficulty level")

    join_df = df[join_mask]
    return join_df.sample(n=number_of_samples, random_state=1)

df_copy = add_length_column(df_copy)
df_copy.head()

c:\Users\lauth\OneDrive\Desktop\azure-vm\Repo\azure-vm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,instruction,input,response,source,text,total_length
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...,sql_create_context,Below are sql tables schemas paired with instr...,25
3,List the record of 0-1 from the table?,CREATE TABLE table_14656147_2 (\n week VARC...,SELECT week FROM table_14656147_2 WHERE record...,sql_create_context,Below are sql tables schemas paired with instr...,25
4,"Which silver has a Gold smaller than 12, a Ran...",CREATE TABLE table_name_24 (\n silver VARCH...,SELECT silver FROM table_name_24 WHERE gold < ...,sql_create_context,Below are sql tables schemas paired with instr...,47
7,Name the most 3 credits,CREATE TABLE table_148535_2 (\n Id VARCHAR\n),SELECT MIN(3 AS _credits) FROM table_148535_2,sql_create_context,Below are sql tables schemas paired with instr...,18
9,"In what years was there a rank lower than 9, u...",CREATE TABLE table_name_63 (\n years VARCHA...,SELECT years FROM table_name_63 WHERE matches ...,sql_create_context,Below are sql tables schemas paired with instr...,47


In [2]:
lvl1_df = filter_by_join_count(df_copy, "lvl1", 20)
lvl2_df = filter_by_join_count(df_copy, "lvl2", 20)
lvl3_df = filter_by_join_count(df_copy, "lvl3", 20)
lvl4_df = filter_by_join_count(df_copy, "lvl4", 20)

responses_lvl1 = lvl1_df["response"].tolist()
responses_lvl2 = lvl2_df["response"].tolist()
responses_lvl3 = lvl3_df["response"].tolist()
responses_lvl4 = lvl4_df["response"].tolist()

context_lvl1 = lvl1_df["input"].tolist()
context_lvl2 = lvl2_df["input"].tolist()
context_lvl3 = lvl3_df["input"].tolist()
context_lvl4 = lvl4_df["input"].tolist()

In [3]:
from train.mistral7bbase.functions.data_functions import get_gpt_relationships_prompt
import pyperclip

p = get_gpt_relationships_prompt(responses_lvl4)
# pyperclip.copy(p)
print(p)

You will be given a list of SQL QUERYS
Your task is to find JOIN relationships in each SQL QUERY.

FOLLOW THIS EXAMPLE FOR EACH SQL QUERY:
SQL QUERY:
SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id = T2.department_id JOIN head AS T3 ON T2.head_id = T3.head_id WHERE T3.born_state = 'Alabama'

ANSWER FORMAT FOR EACH SQL QUERY:
answer = [
...,
'-- department.department_id can be joined with management.department_id, -- management.head_id can be joined with head.head_id',
...
]

YOUR FINAL ANSWER MUST BE A COMMA SEPARATED LIST OF THE ANSWERS OF EACH SQL QUERY

SQL QUERY 1: SELECT T2.name_first, T2.name_last FROM salary AS T1 JOIN player AS T2 ON T1.player_id = T2.player_id JOIN team AS T3 ON T3.team_id_br = T1.team_id WHERE T1.year = 2005 AND T3.name = 'Washington Nationals' INTERSECT SELECT T2.name_first, T2.name_last FROM salary AS T1 JOIN player AS T2 ON T1.player_id = T2.player_id JOIN team AS T3 ON T3.team_id_br = T1.team_id WHERE T1.year = 

In [4]:
gpt_relations_lvl1 = [
    '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found', '-- No JOIN relationships found'
    ]

gpt_relations_lvl2 = [
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- university.school_id can be joined with basketball_match.school_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- artists.id can be joined with albums.artist_id',
'-- customers.customer_id can be joined with customer_orders.customer_id',
'-- demographic.hadm_id can be joined with procedures.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with lab.hadm_id',
'-- bank.branch_id can be joined with loan.branch_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id'
]

gpt_relations_lvl3 = [
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- participates_in.actid can be joined with activity.actid, -- participates_in.actid can be joined with activity.actid',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- student.stuid can be joined with lives_in.stuid, -- lives_in.dormid can be joined with dorm.dormid',
'-- campuses.id can be joined with faculty.campus, -- campuses.id can be joined with degrees.campus',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- exhibition_record.exhibition_id can be joined with exhibition.exhibition_id',
'-- physician.employeeid can be joined with prescribes.physician, -- prescribes.medication can be joined with medication.code',
'-- demographic.hadm_id can be joined with prescriptions.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- enrolled_in.stuid can be joined with student.StuID, -- gradeconversion.lettergrade can be joined with enrolled_in.Grade',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with lab.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- demographic.hadm_id can be joined with diagnoses.hadm_id, -- demographic.hadm_id can be joined with prescriptions.hadm_id',
'-- papers.paperid can be joined with authorship.paperid, -- authorship.instid can be joined with inst.instid',
'-- customers.customer_id can be joined with customer_orders.customer_id, -- customer_orders.order_id can be joined with order_items.order_id',
'-- patient.SSN can be joined with prescribes.patient, -- prescribes.physician can be joined with physician.employeeid'
]

gpt_relations_lvl4 = [
'-- salary.player_id can be joined with player.player_id, -- salary.team_id can be joined with team.team_id_br',
'-- performance.bandmate can be joined with band.id, -- songs.songid can be joined with performance.songid, -- songs.songid can be joined with instruments.songid, -- band.id can be joined with instruments.bandmateid',
'-- employee.eid can be joined with certificate.eid, -- aircraft.aid can be joined with certificate.aid',
'-- tasks.project_id can be joined with projects.project_id, -- tasks.project_id can be joined with project_outcomes.project_id',
'-- customers.customer_id can be joined with orders.customer_id, -- orders.order_id can be joined with order_items.order_id, -- order_items.product_id can be joined with products.product_id',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlist_tracks.playlist_id can be joined with playlists.id',
'-- swimmer.id can be joined with record.swimmer_id, -- record.event_id can be joined with event.id, -- event.stadium_id can be joined with stadium.id',
'-- Tourist_Attractions.Tourist_Attraction_ID can be joined with VISITS.Tourist_Attraction_ID, -- VISITORS.Tourist_ID can be joined with VISITS.Tourist_ID',
'-- problems.product_id can be joined with product.product_id, -- problems.reported_by_staff_id can be joined with staff.staff_id',
'-- tracks.id can be joined with invoice_lines.track_id, -- invoices.id can be joined with invoice_lines.invoice_id, -- invoices.customer_id can be joined with customers.id',
'-- employee.emp_num can be joined with professor.emp_num, -- professor.dept_code can be joined with department.dept_code',
'-- class.class_code can be joined with enroll.class_code, -- class.crs_code can be joined with course.crs_code, -- course.dept_code can be joined with department.dept_code',
'-- tracks.id can be joined with playlist_tracks.track_id, -- playlist_tracks.playlist_id can be joined with playlists.id',
'-- personfriend.name can be joined with person.name, -- personfriend.friend can be joined with personfriend.name, -- personfriend.friend can be joined with personfriend.name',
'-- affected_region.region_id can be joined with region.region_id, -- affected_region.storm_id can be joined with storm.storm_id',
'-- Album.AlbumId can be joined with Track.AlbumId, -- Track.GenreID can be joined with Genre.GenreID',
'-- assignedto.project can be joined with projects.code, -- assignedto.scientist can be joined with scientists.ssn',
'-- student.stuid can be joined with lives_in.stuid, -- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid',
'-- dorm.dormid can be joined with has_amenity.dormid, -- has_amenity.amenid can be joined with dorm_amenity.amenid',
'-- races.raceid can be joined with results.raceid, -- results.driverid can be joined with drivers.driverid'
]


In [21]:
from train.mistral7bbase.functions.data_functions import get_gpt_comments_prompt
import pyperclip

p = get_gpt_comments_prompt(context_lvl2[:1])
p = get_gpt_comments_prompt(context_lvl2[1:3])
p = get_gpt_comments_prompt(context_lvl2[3:6])
p = get_gpt_comments_prompt(context_lvl2[6:10])
p = get_gpt_comments_prompt(context_lvl2[10:15])
p = get_gpt_comments_prompt(context_lvl2[15:20])

p = get_gpt_comments_prompt(context_lvl3[:1])
# p = get_gpt_comments_prompt(context_lvl3[1:3])
# p = get_gpt_comments_prompt(context_lvl3[3:6])
# p = get_gpt_comments_prompt(context_lvl3[6:10])
# p = get_gpt_comments_prompt(context_lvl3[10:15])
# p = get_gpt_comments_prompt(context_lvl3[15:20])
pyperclip.copy(p)
# print(p)

In [10]:
answer = [
    '''
    CREATE TABLE IF NOT EXISTS table_19662262_6 (
        bbm VARCHAR, -- BBM (Blackberry Messenger) ID
        strike_rate VARCHAR -- Strike rate in cricket
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_26 (
        max_1_min_wind_mph__km_h_ VARCHAR, -- Maximum wind speed in mph or km/h
        dates_active VARCHAR -- Dates when the record is active
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_11960944_11 (
        location_attendance VARCHAR, -- Location of attendance
        high_assists VARCHAR -- High assists in a game
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of charting
        flag text, -- Indicator flag
        value_unit text, -- Unit of value
        label text, -- Label information
        fluid text -- Type of fluid
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of diagnosis
        long_title text -- Long title of diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of drug
        drug text, -- Drug name
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Drug dosage
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of procedure
        long_title text -- Long title of procedure
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        name text, -- Name of patient
        marital_status text, -- Marital status of patient
        age text, -- Age of patient
        dob text, -- Date of birth of patient
        gender text, -- Gender of patient
        language text, -- Language spoken by patient
        religion text, -- Religion of patient
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of patient
        expire_flag text, -- Flag indicating expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Admission time
        dischtime text, -- Discharge time
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_16225511_2 (
        school VARCHAR, -- School name
        percent VARCHAR -- Percentage
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_96 (
        school VARCHAR, -- School name
        player VARCHAR -- Player name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_27225944_3 (
        outgoing_manager VARCHAR, -- Name of outgoing manager
        manner_of_departure VARCHAR, -- Manner of departure
        position_in_table VARCHAR -- Position in the league table
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_24706337_1 (
        victor VARCHAR, -- Victor's name
        name_of_the_war VARCHAR -- Name of the war
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_16295365_1 (
        conference VARCHAR, -- Conference name
        school VARCHAR -- School name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_2051288_1 (
        _2000_ VARCHAR, -- Value for the year 2000
        population__may INTEGER, -- Population in May
        location VARCHAR, -- Location
        barangay VARCHAR -- Barangay name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_57 (
        rank INTEGER, -- Rank
        rider VARCHAR -- Rider name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        itemid text, -- Item ID
        charttime text, -- Time of charting
        flag text, -- Indicator flag
        value_unit text, -- Unit of value
        label text, -- Label information
        fluid text -- Type of fluid
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of diagnosis
        long_title text -- Long title of diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        name text, -- Name of patient
        marital_status text, -- Marital status of patient
        age text, -- Age of patient
        dob text, -- Date of birth of patient
        gender text, -- Gender of patient
        language text, -- Language spoken by patient
        religion text, -- Religion of patient
        admission_type text, -- Type of admission
        days_stay text, -- Number of days stayed
        insurance text, -- Insurance information
        ethnicity text, -- Ethnicity of patient
        expire_flag text, -- Flag indicating expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Admission time
        dischtime text, -- Discharge time
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icd9_code text, -- ICD-9 code
        short_title text, -- Short title of procedure
        long_title text -- Long title of procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Subject ID
        hadm_id text, -- Hospital admission ID
        icustay_id text, -- ICU stay ID
        drug_type text, -- Type of drug
        drug text, -- Drug name
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Drug dosage
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_17 (
        wins VARCHAR, -- Number of wins
        points INTEGER -- Points
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_89 (
        one_way VARCHAR, -- Direction
        miles_ INTEGER, -- Miles traveled
        fans_took VARCHAR -- Number of fans took
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_13770460_3 (
        eu VARCHAR, -- European Union abbreviation
        country VARCHAR -- Country name
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS party (
        party_id number, -- Party ID
        party_theme text, -- Party theme
        location text, -- Location of party
        first_year text, -- First year of party
        last_year text, -- Last year of party
        number_of_hosts number -- Number of hosts
    );
    CREATE TABLE IF NOT EXISTS host (
        host_id number, -- Host ID
        name text, -- Name of host
        nationality text, -- Nationality of host
        age text -- Age of host
    );
    CREATE TABLE IF NOT EXISTS party_host (
        party_id number, -- Party ID
        host_id number, -- Host ID
        is_main_in_charge others -- Indicator for main host
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS participants (
        participant_type_code VARCHAR -- Participant type code
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_80 (
        score VARCHAR, -- Score
        year VARCHAR -- Year
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS table_name_99 (
        spine VARCHAR, -- Spine
        number_of_issues VARCHAR -- Number of issues
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS trip (
        id VARCHAR, -- Trip ID
        duration VARCHAR -- Duration of trip
    );
    '''
]


In [8]:
answer = [
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    '''
]

answer = [
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    '''
]

answer = [
    '''
    CREATE TABLE IF NOT EXISTS basketball_match (
        team_name VARCHAR, -- Name of the basketball team
        school_id VARCHAR -- Identifier for the school
    );
    CREATE TABLE IF NOT EXISTS university (
        school_id VARCHAR -- Identifier for the school
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    '''
]

answer = [
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    '''
]

answer = [
    '''
    CREATE TABLE IF NOT EXISTS playlist_tracks (
        playlist_id number, -- Identifier for the playlist
        track_id number -- Identifier for the track
    );
    CREATE TABLE IF NOT EXISTS customers (
        id number, -- Identifier for the customer
        first_name text, -- First name of the customer
        last_name text, -- Last name of the customer
        company text, -- Company of the customer
        address text, -- Address of the customer
        city text, -- City of the customer
        state text, -- State of the customer
        country text, -- Country of the customer
        postal_code text, -- Postal code of the customer
        phone text, -- Phone number of the customer
        fax text, -- Fax number of the customer
        email text, -- Email address of the customer
        support_rep_id number -- Identifier for the support representative
    );
    CREATE TABLE IF NOT EXISTS albums (
        id number, -- Identifier for the album
        title text, -- Title of the album
        artist_id number -- Identifier for the artist
    );
    CREATE TABLE IF NOT EXISTS tracks (
        id number, -- Identifier for the track
        name text, -- Name of the track
        album_id number, -- Identifier for the album
        media_type_id number, -- Identifier for the media type
        genre_id number, -- Identifier for the genre
        composer text, -- Composer of the track
        milliseconds number, -- Length of the track in milliseconds
        bytes number, -- Size of the track in bytes
        unit_price number -- Unit price of the track
    );
    CREATE TABLE IF NOT EXISTS invoices (
        id number, -- Identifier for the invoice
        customer_id number, -- Identifier for the customer
        invoice_date time, -- Date of the invoice
        billing_address text, -- Billing address for the invoice
        billing_city text, -- Billing city for the invoice
        billing_state text, -- Billing state for the invoice
        billing_country text, -- Billing country for the invoice
        billing_postal_code text, -- Billing postal code for the invoice
        total number -- Total amount for the invoice
    );
    CREATE TABLE IF NOT EXISTS artists (
        id number, -- Identifier for the artist
        name text -- Name of the artist
    );
    CREATE TABLE IF NOT EXISTS employees (
        id number, -- Identifier for the employee
        last_name text, -- Last name of the employee
        first_name text, -- First name of the employee
        title text, -- Title of the employee
        reports_to number, -- Identifier for the employee's supervisor
        birth_date time, -- Birth date of the employee
        hire_date time, -- Hire date of the employee
        address text, -- Address of the employee
        city text, -- City of the employee
        state text, -- State of the employee
        country text, -- Country of the employee
        postal_code text, -- Postal code of the employee
        phone text, -- Phone number of the employee
        fax text, -- Fax number of the employee
        email text -- Email address of the employee
    );
    CREATE TABLE IF NOT EXISTS sqlite_sequence (
        name text, -- Name of the sequence
        seq text -- Current value of the sequence
    );
    CREATE TABLE IF NOT EXISTS playlists (
        id number, -- Identifier for the playlist
        name text -- Name of the playlist
    );
    CREATE TABLE IF NOT EXISTS genres (
        id number, -- Identifier for the genre
        name text -- Name of the genre
    );
    CREATE TABLE IF NOT EXISTS media_types (
        id number, -- Identifier for the media type
        name text -- Name of the media type
    );
    CREATE TABLE IF NOT EXISTS invoice_lines (
        id number, -- Identifier for the invoice line
        invoice_id number, -- Identifier for the invoice
        track_id number, -- Identifier for the track
        unit_price number, -- Unit price of the track
        quantity number -- Quantity of the track
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS suppliers (
        supplier_id number, -- Identifier for the supplier
        supplier_name text, -- Name of the supplier
        supplier_phone text -- Phone number of the supplier
    );
    CREATE TABLE IF NOT EXISTS addresses (
        address_id number, -- Identifier for the address
        address_details text -- Details of the address
    );
    CREATE TABLE IF NOT EXISTS customer_addresses (
        customer_id number, -- Identifier for the customer
        address_id number, -- Identifier for the address
        date_from time, -- Start date for the address
        date_to time -- End date for the address
    );
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id number, -- Identifier for the order item
        order_id number, -- Identifier for the order
        product_id number -- Identifier for the product
    );
    CREATE TABLE IF NOT EXISTS department_store_chain (
        dept_store_chain_id number, -- Identifier for the department store chain
        dept_store_chain_name text -- Name of the department store chain
    );
    CREATE TABLE IF NOT EXISTS department_stores (
        dept_store_id number, -- Identifier for the department store
        dept_store_chain_id number, -- Identifier for the department store chain
        store_name text, -- Name of the department store
        store_address text, -- Address of the department store
        store_phone text, -- Phone number of the department store
        store_email text -- Email address of the department store
    );
    CREATE TABLE IF NOT EXISTS customers (
        customer_id number, -- Identifier for the customer
        payment_method_code text, -- Code for the payment method
        customer_code text, -- Code for the customer
        customer_name text, -- Name of the customer
        customer_address text, -- Address of the customer
        customer_phone text, -- Phone number of the customer
        customer_email text -- Email address of the customer
    );
    CREATE TABLE IF NOT EXISTS supplier_addresses (
        supplier_id number, -- Identifier for the supplier
        address_id number, -- Identifier for the address
        date_from time, -- Start date for the address
        date_to time -- End date for the address
    );
    CREATE TABLE IF NOT EXISTS staff_department_assignments (
        staff_id number, -- Identifier for the staff member
        department_id number, -- Identifier for the department
        date_assigned_from time, -- Start date for the assignment
        job_title_code text, -- Code for the job title
        date_assigned_to time -- End date for the assignment
    );
    CREATE TABLE IF NOT EXISTS staff (
        staff_id number, -- Identifier for the staff member
        staff_gender text, -- Gender of the staff member
        staff_name text -- Name of the staff member
    );
    CREATE TABLE IF NOT EXISTS product_suppliers (
        product_id number, -- Identifier for the product
        supplier_id number, -- Identifier for the supplier
        date_supplied_from time, -- Start date for the supply
        date_supplied_to time, -- End date for the supply
        total_amount_purchased text, -- Total amount purchased of the product
        total_value_purchased number -- Total value purchased of the product
    );
    CREATE TABLE IF NOT EXISTS customer_orders (
        order_id number, -- Identifier for the order
        customer_id number, -- Identifier for the customer
        order_status_code text, -- Code for the order status
        order_date time -- Date of the order
    );
    CREATE TABLE IF NOT EXISTS departments (
        department_id number, -- Identifier for the department
        dept_store_id number, -- Identifier for the department store
        department_name text -- Name of the department
    );
    CREATE TABLE IF NOT EXISTS products (
        product_id number, -- Identifier for the product
        product_type_code text, -- Code for the product type
        product_name text, -- Name of the product
        product_price number -- Price of the product
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        itemid text, -- Identifier for the lab test
        charttime text, -- Time the lab test was performed
        flag text, -- Flag for abnormal values
        value_unit text, -- Unit of measurement for the lab value
        label text, -- Label for the lab test
        fluid text -- Type of fluid tested
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the diagnosis
        short_title text, -- Short title for the diagnosis
        long_title text -- Long title for the diagnosis
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icustay_id text, -- ICU stay identifier
        drug_type text, -- Type of drug prescribed
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Formulary drug code
        route text, -- Route of administration
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        name text, -- Patient's name
        marital_status text, -- Patient's marital status
        age text, -- Patient's age
        dob text, -- Patient's date of birth
        gender text, -- Patient's gender
        language text, -- Patient's language
        religion text, -- Patient's religion
        admission_type text, -- Type of admission
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Patient's insurance
        ethnicity text, -- Patient's ethnicity
        expire_flag text, -- Flag for patient's expiration
        admission_location text, -- Location of admission
        discharge_location text, -- Location of discharge
        diagnosis text, -- Diagnosis
        dod text, -- Date of death
        dob_year text, -- Year of birth
        dod_year text, -- Year of death
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Hospital admission identifier
        icd9_code text, -- ICD-9 code for the procedure
        short_title text, -- Short title for the procedure
        long_title text -- Long title for the procedure
    );
    '''
]

answer = [
    '''
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );

    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );

    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );

    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );

    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS bank (
        bname VARCHAR, -- Name of the bank
        branch_id VARCHAR -- Identifier for a bank branch
    );

    CREATE TABLE IF NOT EXISTS loan (
        branch_id VARCHAR -- Identifier for a bank branch
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS demographic (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        name text, -- Name of the patient
        marital_status text, -- Marital status of the patient
        age text, -- Age of the patient
        dob text, -- Date of birth of the patient
        gender text, -- Gender of the patient
        language text, -- Language spoken by the patient
        religion text, -- Religion of the patient
        admission_type text, -- Type of admission (e.g. emergency, elective)
        days_stay text, -- Length of stay in the hospital
        insurance text, -- Type of insurance (e.g. Medicare, private)
        ethnicity text, -- Ethnicity of the patient
        expire_flag text, -- Flag indicating if the patient expired in the hospital
        admission_location text, -- Location of admission (e.g. ER, ICU)
        discharge_location text, -- Location of discharge (e.g. home, nursing home)
        diagnosis text, -- Diagnosis at discharge
        dod text, -- Date of death (if applicable)
        dob_year text, -- Year of birth (derived from dob)
        dod_year text, -- Year of death (derived from dod)
        admittime text, -- Time of admission
        dischtime text, -- Time of discharge
        admityear text -- Year of admission (derived from admittime)
    );
    CREATE TABLE IF NOT EXISTS lab (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        itemid text, -- Identifier for a lab test item
        charttime text, -- Time the lab test was performed
        flag text, -- Flag indicating abnormal test result
        value_unit text, -- Unit of measurement for the test result
        label text, -- Label for the test result
        fluid text -- Type of fluid tested (e.g. blood, urine)
    );
    CREATE TABLE IF NOT EXISTS procedures (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis or procedure
        short_title text, -- Short description of the diagnosis or procedure
        long_title text -- Long description of the diagnosis or procedure
    );
    CREATE TABLE IF NOT EXISTS prescriptions (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icustay_id text, -- Identifier for an ICU stay
        drug_type text, -- Type of drug (e.g. antibiotic, pain medication)
        drug text, -- Name of the drug
        formulary_drug_cd text, -- Code for the drug in the hospital's formulary
        route text, -- Route of administration (e.g. oral, IV)
        drug_dose text -- Dose of the drug
    );
    CREATE TABLE IF NOT EXISTS diagnoses (
        subject_id text, -- Unique identifier for a patient
        hadm_id text, -- Identifier for a hospital admission
        icd9_code text, -- Code for the diagnosis
        short_title text, -- Short description of the diagnosis
        long_title text -- Long description of the diagnosis
    );
    '''
]


In [11]:
len(answer)

20